In [4]:
import numpy as np
from scipy.stats import poisson, weibull_min, norm, gamma
from functools import reduce

## Q1

In [5]:
lambda_val = 2.9
k = 0.8
sigma = 3
total_rainfall_threshold = 5
total_samples = 0
np.random.seed(210123043)

In [6]:
P_S0 = poisson.pmf(0, lambda_val)

In [7]:
def monte_carlo_simulation(num_trials):
    count_below_threshold = 0

    for _ in range(num_trials):
        S = poisson.rvs(lambda_val)

        rainfall_depths = weibull_min.rvs(k, scale=sigma, size=S)

        total_rainfall = np.sum(rainfall_depths)

        if total_rainfall < total_rainfall_threshold:
            count_below_threshold += 1

    probability = count_below_threshold / num_trials
    return probability

In [8]:
def stratification_simulation(num_trials):
    strata = [0, 1, 2, 3, 4, 5, 6]
    probabilities = []
    total_samples = 0
    for S in strata:
        stratum_probability = poisson.pmf(S, lambda_val)

        count_below_threshold = 0
        coun = num_trials*((stratum_probability)/(1-P_S0))
        if(S == 0):
            coun = 2
            total_samples+=2
        else:
            if(S != 6):
                coun = num_trials*((stratum_probability)/(1-P_S0))
                coun = np.floor(coun)
                coun = int(coun)
                total_samples+=coun
        if(S != 6):
            print(f"For s = {S}, number of samples = {coun}")
        else:
            coun = num_trials - total_samples
            print(f"For s>=6, number of samples = {coun}")
        for _ in range(coun):
            rainfall_depths = weibull_min.rvs(k, scale=sigma, size=S)

            total_rainfall = np.sum(rainfall_depths)

            if total_rainfall < total_rainfall_threshold:
                count_below_threshold += 1

        stratum_probability *= count_below_threshold / coun
        probabilities.append(stratum_probability)

    overall_probability = sum(probabilities)
    return overall_probability

In [9]:
cases = [100, 10000]
z_value = norm.ppf(0.995)

In [10]:
print("--------------\nMonte Carlo")
for n in cases:
    print(f"\nFor {n} samples:")
    p = monte_carlo_simulation(n)
    l = p - z_value * np.sqrt(p * (1 - p) / n)
    r = p + z_value * np.sqrt(p * (1 - p) / n)
    print(f"Estimated probability: {p}, and 99% confidence interval: {l, r}")

print("--------------\n")

--------------
Monte Carlo

For 100 samples:
Estimated probability: 0.33, and 99% confidence interval: (0.20888123025369726, 0.4511187697463028)

For 10000 samples:
Estimated probability: 0.3747, and 99% confidence interval: (0.3622318177889534, 0.38716818221104654)
--------------



In [11]:
print("--------------\nStratification")
for n in cases:
    print(f"\nFor {n} samples:")
    p = stratification_simulation(n)
    l = p - z_value * np.sqrt(p * (1 - p) / n)
    r = p + z_value * np.sqrt(p * (1 - p) / n)
    print(f"Estimated probability: {p}, and 99% confidence interval: {l, r}")

print("--------------\n")

--------------
Stratification

For 100 samples:
For s = 0, number of samples = 2
For s = 1, number of samples = 16
For s = 2, number of samples = 24
For s = 3, number of samples = 23
For s = 4, number of samples = 17
For s = 5, number of samples = 9
For s>=6, number of samples = 9
Estimated probability: 0.3491716407554088, and 99% confidence interval: (0.226379719171599, 0.4719635623392186)

For 10000 samples:
For s = 0, number of samples = 2
For s = 1, number of samples = 1688
For s = 2, number of samples = 2448
For s = 3, number of samples = 2366
For s = 4, number of samples = 1715
For s = 5, number of samples = 995
For s>=6, number of samples = 786
Estimated probability: 0.38534055773506365, and 99% confidence interval: (0.37280462309409307, 0.39787649237603423)
--------------



## Q2

In [12]:
alpha_lst = [2082, 1999, 2008, 2047, 2199, 2153, 1999, 2136, 2053, 2121, 1974, 2110, 2110, 2168, 2035, 2019, 2044, 2191, 2284, 1912, 2196, 2099, 2041, 2192, 2188, 1984, 2158, 2019, 2032, 2051, 2192, 2133, 2142, 2113, 2150, 2221, 2046, 2127]
prob_lst = []
N = 10000
for _ in range(N):
    y19 = np.random.gamma(shape = alpha_lst[18], scale = 1, size = 1)
    p = 1
    cdfs_product = 1
    for i in range(38):
        if i != 18:
            cdfs_product *= gamma.cdf(y19, a=alpha_lst[i])

    p = cdfs_product
    prob_lst.append(p)
mu = np.array(prob_lst).mean()
print(f"The required probablity is {mu}")

The required probablity is 0.6288936628724892


## Q3

In [13]:
mu = [0, 0, 0, 0, 0]
var = [5, 5, 5, 5, 5]

In [14]:
def generate_lognormal(mu, var, size):
    normal_random_variables = mu + np.sqrt(var) * np.random.randn(size)
    lognormal_random_variables = np.exp(normal_random_variables)
    return lognormal_random_variables

In [15]:
X = []
samples = 100000
for i in range(5):
    X.append(generate_lognormal(mu[i], var[i], samples))
X = np.array(X)
f = np.sum(X, axis = 0) / 5
h = np.prod(X, axis = 0) / 5

theta = np.exp(sum(mu) + 0.5 * sum(var)) / 5

theta_hat = np.mean(h)
mu_hat = np.mean(f)

beta_opt = np.sum((f - mu_hat)*(h - theta_hat)) / np.sum(np.square(h - theta_hat))
print(f" Beta optimal: {beta_opt}")

mean_hat = mu_hat + beta_opt * (theta - theta_hat)
var_hat = np.sum(np.square(f - mu_hat - beta_opt * (h - theta_hat))) / (n**2)
z_value = norm.ppf(0.995)
l = mean_hat - z_value*np.sqrt(var_hat)
r = mean_hat + z_value*np.sqrt(var_hat)
print(f"For {n} samples, Estimated mean = {mean_hat} with 99% confidence interval - ({l}, {r})")
print(f"With Parameters: ({mu[0]}, {var[0]}), ({mu[1]}, {var[1]}), ({mu[2]}, {var[2]}), ({mu[3]}, {var[3]}), ({mu[4]}, {var[4]})")
print()

 Beta optimal: 1.4666680224904472e-06
For 10000 samples, Estimated mean = 12.481561614254415 with 99% confidence interval - (8.544610212439386, 16.418513016069443)
With Parameters: (0, 5), (0, 5), (0, 5), (0, 5), (0, 5)

